In [1]:
# designer : bae hueng myoung(eng name: benjamin)
# email : naroo135@gmail.com
# prj : classifier design(class 3: rock paper scissor)
# prj golas : 
    #1. classifier desgin
    #2. dealing with overfitting
    #3. test accuarcy more than 60%

# prj flow diagram

    
    
    
    

############## required modules ##############
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import os, glob


############## required functions ##############
# resize function
def resize_images(img_path):
	images=glob.glob(img_path + "/*.jpg")  
    
	print(len(images), " images to be resized.")

    # 파일마다 모두 28x28 사이즈로 바꾸어 저장합니다.
	target_size=(28,28)
	for img in images:
		old_img=Image.open(img)
		new_img=old_img.resize(target_size,Image.ANTIALIAS)
		new_img.save(img, "JPEG")
	print(len(images), " images resized.")

    
# load data and labeling
def load_data(img_path, number_of_data=300):  # 가위바위보 이미지 개수 총합에 주의하세요.
    # 가위 : 0, 바위 : 1, 보 : 2
    img_size=28
    color=3
    #이미지 데이터와 라벨(가위 : 0, 바위 : 1, 보 : 2) 데이터를 담을 행렬(matrix) 영역을 생성합니다.
    imgs=np.zeros(number_of_data*img_size*img_size*color,dtype=np.int32).reshape(number_of_data,img_size,img_size,color)
    labels=np.zeros(number_of_data,dtype=np.int32)

    idx=0
    for file in glob.iglob(img_path+'/new_scissor/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=0   # 가위 : 0
        idx=idx+1

    for file in glob.iglob(img_path+'/new_rock/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=1   # 바위 : 1
        idx=idx+1  
    
    for file in glob.iglob(img_path+'/new_paper/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=2   # 보 : 2
        idx=idx+1
        
    print("학습데이터(x_train)의 이미지 개수는", idx,"입니다.")
    return imgs, labels


# model design function
def model_design(n_channel_1,n_channel_2,n_dense,class_num):
    model=keras.models.Sequential()
    model.add(keras.layers.Conv2D(n_channel_1, (3,3), activation='relu', input_shape=(28,28,3)))
    model.add(keras.layers.MaxPool2D(2,2))
    model.add(keras.layers.Conv2D(n_channel_2, (3,3), activation='relu'))
    model.add(keras.layers.MaxPooling2D((2,2)))
    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(n_dense, activation='relu'))
    model.add(keras.layers.Dense(class_num, activation='softmax'))

    model.summary()
    model.compile(optimizer='adam',
                 loss='sparse_categorical_crossentropy',
                 metrics=['accuracy'])

    return model


############## work flow ##############
# resize work
''' 기존 훈련 데이터 셋
image_dir_path = os.getenv("HOME") + "/aiffel/Basic_DeepLearningNetwork/scissor"
resize_images(image_dir_path)

image_dir_path = os.getenv("HOME") + "/aiffel/Basic_DeepLearningNetwork/rock"
resize_images(image_dir_path)

image_dir_path = os.getenv("HOME") + "/aiffel/Basic_DeepLearningNetwork/paper"
resize_images(image_dir_path)
'''

image_dir_path = os.getenv("HOME") + "/aiffel/Basic_DeepLearningNetwork/new_rsp/new_scissor"
resize_images(image_dir_path)

image_dir_path = os.getenv("HOME") + "/aiffel/Basic_DeepLearningNetwork/new_rsp/new_rock"
resize_images(image_dir_path)

image_dir_path = os.getenv("HOME") + "/aiffel/Basic_DeepLearningNetwork/new_rsp/new_paper"
resize_images(image_dir_path)




# normalization and labeling work
total_img_num = 3000
image_dir_path = os.getenv("HOME") + "/aiffel/Basic_DeepLearningNetwork/new_rsp"
(x_train, y_train)=load_data(image_dir_path,total_img_num)
x_train_norm = x_train/255.0   # 입력은 0~1 사이의 값으로 정규화
print("x_train shape: {}".format(x_train.shape))
print("y_train shape: {}".format(y_train.shape))

# channel configuration work
print("Before Reshape - x_train_norm shape: {}".format(x_train_norm.shape))
#print("Before Reshape - x_test_norm shape: {}".format(x_test_norm.shape))
x_train_reshaped=x_train_norm.reshape( -1, 28, 28, 3)  # 데이터갯수에 -1을 쓰면 reshape시 자동계산됩니다.
#x_test_reshaped=x_test_norm.reshape( -1, 28, 28, 1)
print("After Reshape - x_train_reshaped shape: {}".format(x_train_reshaped.shape))
#print("After Reshape - x_test_reshaped shape: {}".format(x_test_reshaped.shape))

print(len(x_train_norm))
print(len(x_train_reshaped))


# model design work
n_channel_1=16
n_channel_2=32
n_dense=64
n_train_epoch=10
class_num = 3
model = model_design(n_channel_1,n_channel_2,n_dense,class_num)

# model trainning work
model.fit(x_train_reshaped, y_train, epochs=n_train_epoch)



# evaluation of model test performance 
test_img_path = os.getenv('HOME')+'/aiffel/Basic_DeepLearningNetwork/new_test'
x_test,y_test = load_data(test_img_path, 300)


test_loss, test_accuracy = model.evaluate(x_test, y_test, verbose=2)
print("test_loss: {} ".format(test_loss))
print("test_accuracy: {}".format(test_accuracy))







94/94 [==============================] - 2s 17ms/step - loss: 0.6487 - accuracy: 1.0000
Epoch 6/10
94/94 [==============================] - 2s 17ms/step - loss: 0.5772 - accuracy: 1.0000
Epoch 7/10
94/94 [==============================] - 2s 17ms/step - loss: 0.5145 - accuracy: 1.0000
Epoch 8/10
94/94 [==============================] - 2s 17ms/step - loss: 0.4595 - accuracy: 1.0000
Epoch 9/10
94/94 [==============================] - 2s 17ms/step - loss: 0.4114 - accuracy: 1.0000
Epoch 10/10
94/94 [==============================] - 2s 17ms/step - loss: 0.3693 - accuracy: 1.0000
학습데이터(x_train)의 이미지 개수는 0 입니다.
10/10 - 0s - loss: 0.3498 - accuracy: 1.0000
test_loss: 0.3498004078865051 
test_accuracy: 1.0


In [ ]:

# REPORT
개요 : 
가위바위보 트레인 셋을 각각 1000장씩 찍어서 훈련시켰고 10번의 에포크를 설정하여 훈련시켜본 결과 3번 에포크부터 정확도는 1로써 훈련이 잘 되는 것을 확인할 수 있었다. 하지만 다른 사람의 테스트셋 300장을 테스트해본 결과 정확도는 0.32에 그치고 말았다. 이러한 현상은 오버피팅의 현상으로써 트레인 셋에서 높은 정확도를 자랑하지만 트레이닝 셋과 조금 다른 특징들이 섞여 있는 다른 테스트셋에서는 위와 같은 낮은 정확도의 문제를 야기한다. 이 문제를 해결하기 위해 하이퍼파라미터인 컨벌루션 피쳐갯수와 복잡도갯수 에포크를 각각 변경해 보도록 하였다. 

"어떻게 파라미터를 변경해 볼 것인가?"
테스트 셋에 대한 성능은 훈련된 모델에 따라 달라진다. 특히 하이퍼파라미터인 피쳐갯수, 복잡도와 에포크 수는 테스트 성능을 결정짓는 주요 변수들이다. 먼저 에포크 수를 조정해 보았다. 에포크가 너무 낮으면 훈련의 정확도는 떨어지며 너무 높으면 훈련 내 데이터셋의 정확도는 1에 육박하지만 조금의 다른 특징들이 섞인 테스트셋에서 정확도가 떨어진다. 기본값인 에포크=10을 기준으로 훈련해 본 결과 에포크 3부터 훈련 정확도는 1로 매우 정확하다. 따라서 에포크 3부터 에포크 30까지의 범위를 두고 테스트를 진행해보았다. 수행해 본 결과 0.3 전후로 큰 변화를 나태내지 못하였고 약간의 차이는 있지만 에포크 10에서 가장 좋은 정확도 0.3366을 나타내었다. 따라서 에포크를 10으로 두고 다른 하이퍼파라미터를 변경해 보았다. 복잡도, 피쳐갯수를 2의 배수로 변경해보며 수행해본 결과 정확도는 큰 변화를 나타내지 못하였다. 그리고 어떤 파라미터들이 정확도에 얼마나 연관성이 있는지 일일이 알아보는 것은 매우 난해하고 시간과 비용이 매우 소모되는 일이었다. 

"데이터 문제이지 않을까?"
첫번째 훈련에서 나의 데이터는 카페에서 셔츠와 배경이 다소 난잡한 환경에서 얻은 데이터들이다. 검은색 흰색 체크 남방에 머리카락이 살짝 나온 경우도 있으며 데이터 한장 한장의 복잡도는 가위, 바위, 보의 형상 외에 여러 특징들이 섞여 있다. 더구나 각 클래스별로 1000장씩 훈련시켰기에 배경의 특징까지 모두 학습하여 배경이 거의 없는 테스트 셋에서 좋은 성능을 나타내지 못했을 가능성이 크다. 나는 다시 1000장씩 가위, 바위, 보 형상만 나오도록 촬영하고 최대한 형상의 특징만 나오도록 데이터를 가공하였다. 또한 오버피팅을 최소화하기 위해서 다양한 손모양, 각도들을 촬영하여 다시 훈련을 시도하였다.

"무엇이 문제인가?"
훈련 데이터의 변화 후 모델의 테스트 정확도는 이전과 비교하여 약간의 미동만 있을 뿐이었다. 다른 파라미터를 조율하여도 정확도 0.33에서 크게 벗어나지 않았다. 이것은 목표했던 정확도 60퍼센트와 매우 큰 격차이다. 이 문제를 해결하기 위해서 모델의 알고리즘을 파악할 필요가 있다. 현재까지 딥러닝 네트워크 모델은 단순히 따라하고 파라미터를 임의로 조정하거나 데이터에 변화를 줄 뿐이다. 이번 프로젝트는 최종적으로 완성하진 못했지만 이번 실패는 딥러닝 모델의 알고리즘에 대해 깊이 파고들 수 있는 기폭제로 작용하여 이 문제를 추후 해결해나갈 것이다.
